In [21]:
test_semantic_model_url = 'https://raw.githubusercontent.com/djouallah/semantic_sql_testing/refs/heads/main/semantic_model.txt' 
questions_url           = 'https://raw.githubusercontent.com/djouallah/semantic_sql_testing/refs/heads/main/questions.json'
SF = 1
output_dir="/tmp/llm"

In [22]:
import requests
import json
import duckdb
import pathlib
import time
import pandas as pd
import re
from   openai import AzureOpenAI, OpenAIError
import datetime
import os


# Generate Data

In [23]:
if SF <1 :
 schema = f"{str(SF).replace('.', '_')}"
else :
 schema = f'DS{SF:02d}'
os.makedirs(output_dir, exist_ok=True)
db_path = output_dir +"/"+ schema +".duckdb"
if not pathlib.Path(db_path).exists():
    con = duckdb.connect(db_path)
    con.sql("SET memory_limit = '14GB' ")
    con.sql(f"CALL dsdgen(sf={SF})")
    con.close()
con = duckdb.connect()
con.sql(f""" attach '{db_path}' as ds(read_only) ; use ds """)
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

# Run Tests

In [24]:
def get_ai_response(user_message, LLM, provider, api_key=None, azure_endpoint=None, azure_api_version=None):
    system_prompt = ""
    try:
        github_response = requests.get(test_semantic_model_url)
        github_response.raise_for_status()
        system_prompt = github_response.text.strip()
    except requests.RequestException as e:
        print(f"Error fetching system prompt from GitHub ({test_semantic_model_url}): {e}")
        return f"Error fetching system prompt: {e}"

    generated_text = None
    data = None

    if provider == "ollama":
        url = 'http://127.0.0.1:11434/api/chat'
        headers = {'Content-Type': 'application/json'}
        payload = {
            'model': LLM,
            'messages': [
                {'role': 'system', 'content': system_prompt},
                {'role': 'user', 'content': user_message}
            ],
            'stream': False
        }
        try:
            response = requests.post(url, headers=headers, json=payload)
            response.raise_for_status()
            data = response.json()
            generated_text = data.get('message', {}).get('content', '')
            if generated_text:
                generated_text = generated_text.replace("<think>", "").replace("</think>", "")
        except requests.RequestException as e:
            return f"Error with Ollama API request: {e}"
        except (KeyError, IndexError, AttributeError) as e:
            return f"Unexpected response format from Ollama: {e}\nFull response data: {data}"

    elif provider == "azure_foundry":
        if not azure_endpoint or not azure_api_version or not api_key:
            return "Azure provider requires 'azure_endpoint', 'azure_api_version', and 'api_key' parameters."

        try:
            client = AzureOpenAI(
                api_version=azure_api_version,
                azure_endpoint=azure_endpoint,
                api_key=api_key,
            )

            response = client.chat.completions.create(
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_message}
                ],
                model=LLM,
            )

            generated_text = response.choices[0].message.content

        except Exception as e:
            return f"An error occurred with Azure OpenAI: {e}"



    else:
        return f"Unsupported provider for this test setup: {provider}. Supported providers are 'ollama', 'azure_foundry', and 'gemini'."

    if not isinstance(generated_text, str):
         print(f"Generated text is not a string (type: {type(generated_text)}). Cannot clean.")
         if generated_text is None:
              return "Could not retrieve generated text from provider response."
         else:
              return f"Received unexpected output type from {provider}: {type(generated_text)}. Value: {generated_text}"


    cleaned_text = re.sub(r'```(sql|duckdb)?\s*([\s\S]*?)\s*```', r'\2', generated_text, flags=re.IGNORECASE).strip()

    if "```" in cleaned_text or "SELECT " in cleaned_text.upper() or "WITH " in cleaned_text.upper():
         cleaned_text = cleaned_text.replace("```sql", "").replace("```duckdb", "").replace("```", "").strip()


    return cleaned_text

In [25]:
def execute_sql_with_retry(query, test_model, provider, api_key, azure_endpoint, azure_api_version, max_attempts):
    attempt = 1
    current_query = query.strip()

    while attempt <= max_attempts:
        try:
            result = con.sql(current_query).df()
            return result, attempt, current_query  # ← Always return the last successful query
        except Exception as e:
            error_message = str(e).lower()
            if "syntax" not in error_message and "parser" not in error_message and "binder" not in error_message:
                return f"Non-syntax error: {e}", attempt, current_query

            print(current_query)
            print(f"Attempt {attempt}/{max_attempts} failed with syntax error: {e}")

            if attempt == max_attempts:
                return f"Max attempts reached. Last error: {e}", attempt, current_query

            message = (
                f"The following SQL query has a syntax error: '{current_query}'.\n"
                f"Error message: {e}\n"
                f"Please provide the corrected SQL query. Return only the corrected query without explanation."
            )

            corrected_query = get_ai_response(message, test_model, provider, api_key, azure_endpoint, azure_api_version)

            if corrected_query.startswith("Error"):
                return f"Failed to get corrected query : {corrected_query}", attempt, current_query

            current_query = corrected_query.strip()
            attempt += 1

    return "Unexpected error or loop termination", attempt, current_query

In [26]:
def ask_question(questions, test_model,provider,api_key=None, azure_endpoint=None, azure_api_version=None):
    results_data = [] 
    for i, x in enumerate(questions):
        print(f"Question {i+1}: {x}") # Keep or remove print as needed
        start_time = time.time()
        sql_query_or_error = get_ai_response(x, test_model, provider, api_key, azure_endpoint, azure_api_version)
        print(sql_query_or_error) # Keep or remove print as needed
        query_result_data_json = [] # Initialize as an empty list for JSON result
        attempts_count = None
        error_details = None # Initialize error_details
        if sql_query_or_error is None or sql_query_or_error.startswith("Error"):
            # If get_ai_response returned an error or None, store the error string
            error_message = sql_query_or_error if sql_query_or_error is not None else "AI response was None"
            # print(f"Failed to get query from AI: {error_message}") # Keep or remove print
            # print("Execution: SKIPPED (AI error)") # Keep or remove print
            error_details = f"AI Error: {error_message}" # Store error details
            # query_result_data_json remains empty []
            result_row_count = 0 # Result count is 0 on error
        else:
            result_from_execution, attempts_count,query_returned = execute_sql_with_retry(sql_query_or_error, test_model,provider,api_key, azure_endpoint, azure_api_version, max_attempts=3)

            display(result_from_execution)
            is_successful = isinstance(result_from_execution, pd.DataFrame)

            if is_successful:
                print("Execution: SUCCESS") # Keep or remove print
                query_result_data_json = result_from_execution.to_dict('records')
                error_details = None # No error details on success
                result_row_count = len(result_from_execution) # Calculate row count
            else:
                print("Execution: FAILED") # Keep or remove print
                #query_result_data_json remains empty []
                error_details = f"Execution Error: {result_from_execution}" # Store error details
                result_row_count = 0 # Result count is 0 on failure


        end_time = time.time()
        duration = round(end_time - start_time, 2)
        print(f"\nExecution Time: {duration:.2f} seconds") # Keep or remove print
        print(f" ############################### ") # Keep or remove print
        results_data.append({
            "model" : test_model,
            "timestamp": timestamp,
            "nbr": i + 1,
            "question": x,
            "duration_s": duration, 
            "sql_query": query_returned, 
            "attempts": attempts_count,
            "result": query_result_data_json, 
            "result_count": result_row_count, 
            "error_details": error_details 
        })

    os.makedirs(output_dir, exist_ok=True)
    sanitized_model = re.sub(r'[\\/*?:"<>|]', '_', test_model)
    output_filename = f"{timestamp}_{sanitized_model}.json"
    output_path = os.path.join(output_dir, output_filename)
    try:
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(results_data, f, indent=4)
        return f"Successfully processed {len(questions)} questions. Results saved to {output_path}"
    except IOError as e:
        return f"Error saving results to {output_path}: {e}"
    except Exception as e:
        return f"An unexpected error occurred during file saving: {e}"

In [27]:
try:
    response = requests.get(questions_url)
    response.raise_for_status()  # Raise an exception for HTTP errors (4xx or 5xx)
    questions = json.loads(response.text)

    print("Successfully retrieved questions:")
except requests.exceptions.RequestException as e:
    print(f"Error retrieving file from GitHub: {e}")
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")

Successfully retrieved questions:


In [ ]:
%%time
#ask_question(questions,"o3-mini","azure_foundry",os.getenv("o3_mini_key"), os.getenv("llm_endpoint"),"2025-01-01-preview" )

In [29]:
%%time
#ask_question(questions,"gpt-4o","azure_foundry",os.getenv("LLM_API_KEY"), os.getenv("llm_endpoint"),"2025-01-01-preview" )

CPU times: total: 0 ns
Wall time: 0 ns


In [30]:
%%time
#ask_question(questions,'qwen3:8b',"ollama")

CPU times: total: 0 ns
Wall time: 0 ns


# Check Results

In [31]:
duckdb.sql(f""" from '{output_dir}/*.json' order by nbr,model """).to_view("results")
try:
 duckdb.sql(f""" load excel ;copy results to '{output_dir}/test.xlsx' (format 'xlsx', header 'true', overwrite)  """)
except Exception as e:
    print(f"Error exporting to Excel: {e}")
    print("close the excel file or tey again.")

Error exporting to Excel: IO Error: Extension "C:\Users\mdjouallah\.duckdb\extensions\30e907b52f\windows_amd64\excel.duckdb_extension" not found.
Extension "excel" is an existing extension.

Install it first using "INSTALL excel".
close the excel file or tey again.


In [32]:
# check number of rows and column returned by each model, Null means SQL error
duckdb.sql(f""" 
           pivot( select nbr,question,model,[result_count , cardinality(result[1])] as result_shape from results ) 
           on model using min(result_shape) order by nbr
        """).show(max_width=120)

┌───────┬─────────────────────────────────────────────────────────────┬───────────┬───────────┬────────────┬───────────┐
│  nbr  │                          question                           │  o3-mini  │ qwen3:14b │  qwen3:4b  │ qwen3:8b  │
│ int64 │                           varchar                           │ int128[]  │ int128[]  │  int128[]  │ int128[]  │
├───────┼─────────────────────────────────────────────────────────────┼───────────┼───────────┼────────────┼───────────┤
│     1 │ What is the overall total sales revenue?                    │ [1, 1]    │ [1, 1]    │ [1, 1]     │ [1, 1]    │
│     2 │ What is the total number of items sold across all transac…  │ [1, 1]    │ [1, 1]    │ [1, 1]     │ [1, 1]    │
│     3 │ What is the total monetary value of all returned items?     │ [1, 1]    │ [1, 1]    │ [1, 1]     │ [1, 1]    │
│     4 │ List the names of all stores, order by store name.          │ [12, 1]   │ [12, 1]   │ [12, 1]    │ [12, 1]   │
│     5 │ What are the different

In [33]:
# execusion time in seconds
duckdb.sql(f""" 
           pivot( select nbr,question,model,duration_s from results ) 
           on model using min(duration_s) as duration order by nbr
        """).show(max_width=120)

┌───────┬──────────────────────────────┬──────────────────┬────────────────────┬───────────────────┬───────────────────┐
│  nbr  │           question           │ o3-mini_duration │ qwen3:14b_duration │ qwen3:4b_duration │ qwen3:8b_duration │
│ int64 │           varchar            │      double      │       double       │      double       │      double       │
├───────┼──────────────────────────────┼──────────────────┼────────────────────┼───────────────────┼───────────────────┤
│     1 │ What is the overall total …  │             4.16 │              52.04 │             14.41 │             35.42 │
│     2 │ What is the total number o…  │             3.84 │              15.09 │              1.91 │               9.3 │
│     3 │ What is the total monetary…  │             4.57 │              15.02 │               2.1 │             10.86 │
│     4 │ List the names of all stor…  │             3.51 │              14.88 │              2.35 │              7.08 │
│     5 │ What are the different

In [34]:
different_results = duckdb.sql("""
    SELECT 
        distinct(nbr) 
    FROM results 
    WHERE
    model in ('o3-mini','qwen3:8b') and 
    nbr in
    (
    select nbr from results 
    where model in ('o3-mini','qwen3:8b')
    group by all
    having count(distinct(cardinality(result[1]))) > 
    1 or count(distinct(result_count )) > 1 )
    order by nbr
""").df()['nbr'].tolist()
for nbr in different_results:
    print(f"question {nbr} : " + duckdb.sql(f" select question from results where nbr = {nbr} and model in ('o3-mini') ").fetchone()[0])
    try:
        query = duckdb.sql(f" select sql_query from results where nbr = {nbr} and model in ('o3-mini') ").fetchone()[0]
        df1 = con.sql(query).df()
        query = duckdb.sql(f" select min(sql_query) from results where nbr = {nbr} and model in ('qwen3:8b') ").fetchone()[0]
        df2 = con.sql(query).df()
        side_by_side = pd.concat([df1, df2], axis=1, keys=['DF1', 'DF2'])
        display(side_by_side)
    except Exception as e:
        print(f"Error executing query for nbr {nbr}: {e}")

question 7 : Which store generated the most total sales revenue? (To ensure a deterministic result if there's a tie, you might order by revenue descending and then store name alphabetically).


DF1                        DF2              
  s_store_name   total_sales s_store_name   total_sales
0          bar  8.485581e+08          bar  8.485581e+08
1          NaN           NaN         eing  8.471117e+08
2          NaN           NaN          ese  8.462294e+08
3          NaN           NaN         able  8.459570e+08
4          NaN           NaN        ation  8.452357e+08
5          NaN           NaN        ought  8.437834e+08

question 12 : Calculate the return rate for each item category, ordered alphabetically by item category name.


DF1                       DF2                              \
   item_category return_rate item_category   total_sales total_returns   
0          Books    5.277072         Books  5.031496e+08   26551562.67   
1       Children    5.255010      Children  5.085669e+08   26725240.19   
2    Electronics    5.319352   Electronics  5.132486e+08   27301499.34   
3           Home    5.218650          Home  5.123771e+08   26739164.36   
4        Jewelry    5.345070       Jewelry  4.973734e+08   26584955.23   
5            Men    5.209736           Men  5.091009e+08   26522813.27   
6          Music    5.189974         Music  5.335553e+08   27691381.94   
7          Shoes    5.311438         Shoes  5.266607e+08   27973256.49   
8         Sports    5.320164        Sports  5.113303e+08   27203606.83   
9          Women    5.263331         Women  5.093438e+08   26808449.53   
10          None    0.000000          None  1.395929e+07          0.00   
11          None         NaN          None  0.000000e+00     722679.40   

                
   return_rate  
0     5.277072  
1     5.255010  
2     5.319352  
3     5.218650  
4     5.345070  
5     5.209736  
6     5.189974  
7     5.311438  
8     5.320164  
9     5.263331  
10    0.000000  
11         NaN

question 17 : What is the return rate for items sold on weekends versus weekdays, broken down by customer age groups (e.g., under 30, 30-45, over 45), ordered by age group.


DF1                                                        DF2  \
   age_group day_type   total_sales total_returns return_rate age_group   
0      30-45  Weekday  8.036384e+08   42231793.68    5.255074     20-29   
1      30-45  Weekend  3.256532e+08   16958203.53    5.207442     20-29   
2    Over 45  Weekday  1.691612e+09   89372705.33    5.283286     30-39   
3    Over 45  Weekend  6.773807e+08   35478913.87    5.237663     30-39   
4   Under 30  Weekday  1.106583e+09   56395803.78    5.096389     40-49   
5   Under 30  Weekend  4.411807e+08   23060101.47    5.226906     40-49   
6        NaN      NaN           NaN           NaN         NaN     50-59   
7        NaN      NaN           NaN           NaN         NaN     50-59   
8        NaN      NaN           NaN           NaN         NaN       60+   
9        NaN      NaN           NaN           NaN         NaN       60+   
10       NaN      NaN           NaN           NaN         NaN      < 20   
11       NaN      NaN           NaN           NaN         NaN      < 20   

                                                      
   d_weekend   total_sales total_returns return_rate  
0          Y  2.025206e+08   11042123.56    5.452347  
1          N  5.098070e+08   26393168.04    5.177090  
2          Y  2.044319e+08   10821169.42    5.293287  
3          N  5.036592e+08   26550551.04    5.271531  
4          N  5.053419e+08   26399997.04    5.224185  
5          Y  2.025687e+08   10441801.82    5.154697  
6          Y  2.029946e+08   10674998.32    5.258759  
7          N  5.058094e+08   26516422.79    5.242374  
8          Y  3.410808e+08   17956562.10    5.264606  
9          N  8.573134e+08   45801470.62    5.342442  
10         N  5.967765e+08   30002635.74    5.027449  
11         Y  2.386602e+08   12017977.91    5.035602

question 18 : Which item brand has shown the largest decrease in its return rate when comparing the average rate of 2001 to 2002, specifically for stores located in the 'TN' state? (Order by the decrease in return rate descending, and then by brand name alphabetically for ties).


DF1                                                  \
                     brand return_rate_2001 return_rate_2002 rate_decrease   
0      exportiunivamalg #9        26.432490         5.620388     20.812102   
1         corpunivamalg #3        22.745688         6.939975     15.805714   
2       amalgamalgamalg #5        16.729575         1.143041     15.586534   
3         maxiunivamalg #9        18.323936         2.744060     15.579876   
4    edu packunivamalg #17        17.206425         2.183451     15.022974   
..                     ...              ...              ...           ...   
668                    NaN              NaN              NaN           NaN   
669                    NaN              NaN              NaN           NaN   
670                    NaN              NaN              NaN           NaN   
671                    NaN              NaN              NaN           NaN   
672                    NaN              NaN              NaN           NaN   

                       DF2                                    \
                     brand return_rate_2001 return_rate_2002   
0        corpunivamalg #16     1.199750e+06     1.253507e+05   
1    scholaramalgamalg #16     1.005937e+06     4.797750e+04   
2         maxiunivamalg #4     1.091170e+06     1.427970e+05   
3    scholaramalgamalg #17     1.000791e+06     1.921873e+05   
4      importounivamalg #7     9.837840e+05     1.964542e+05   
..                     ...              ...              ...   
668       namelessbrand #7     2.306338e+05     8.513397e+05   
669     univamalgamalg #10     1.699318e+05     8.407612e+05   
670    exportiunivamalg #4     1.294462e+05     9.452271e+05   
671     corpamalgamalg #12     3.310106e+05     1.160314e+06   
672  scholaramalgamalg #11     1.306130e+05     1.001742e+06   

                             
    decrease_in_return_rate  
0              1.074399e+06  
1              9.579595e+05  
2              9.483730e+05  
3              8.086037e+05  
4              7.873298e+05  
..                      ...  
668           -6.207059e+05  
669           -6.708294e+05  
670           -8.157809e+05  
671           -8.293034e+05  
672           -8.711290e+05  

[673 rows x 8 columns]

question 20 : show all stores net sales , average net sales and percentage of net sales compared to average net sales for each store, ordered by percentage of net sales descending.


DF1                                                          DF2  \
   store_name     net_sales average_net_sales percentage_of_avg s_store_sk   
0         bar  8.044211e+08      4.009164e+08        200.645571         10   
1        eing  8.028028e+08      4.009164e+08        200.241944          8   
2         ese  8.023467e+08      4.009164e+08        200.128165          4   
3        able  8.014186e+08      4.009164e+08        199.896669          2   
4       ation  8.007528e+08      4.009164e+08        199.730609          7   
5       ought  7.992551e+08      4.009164e+08        199.357041          1   
6       ought  0.000000e+00      4.009164e+08          0.000000         11   
7        anti  0.000000e+00      4.009164e+08          0.000000          5   
8        eing  0.000000e+00      4.009164e+08          0.000000          9   
9       ought  0.000000e+00      4.009164e+08          0.000000         12   
10       able  0.000000e+00      4.009164e+08          0.000000          3   
11      cally  0.000000e+00      4.009164e+08          0.000000          6   

                                                               
   s_store_name     net_sales avg_net_sales percentage_of_avg  
0           bar  8.044211e+08  4.009164e+08        200.645571  
1          eing  8.028028e+08  4.009164e+08        200.241944  
2           ese  8.023467e+08  4.009164e+08        200.128165  
3          able  8.014186e+08  4.009164e+08        199.896669  
4         ation  8.007528e+08  4.009164e+08        199.730609  
5         ought  7.992551e+08  4.009164e+08        199.357041  
6         ought  0.000000e+00  4.009164e+08          0.000000  
7          anti  0.000000e+00  4.009164e+08          0.000000  
8          eing  0.000000e+00  4.009164e+08          0.000000  
9         ought  0.000000e+00  4.009164e+08          0.000000  
10         able  0.000000e+00  4.009164e+08          0.000000  
11        cally  0.000000e+00  4.009164e+08          0.000000

In [35]:
good_results = list(set(list(range(1, 21))) - set(different_results))
for nbr in good_results:
    print(f"question {nbr} : " + duckdb.sql(f" select question from results where nbr = {nbr} and model in ('o3-mini') ").fetchone()[0])
    try:
        df1 = con.sql(duckdb.sql(f" select sql_query from results where nbr = {nbr} and model in ('o3-mini') ").fetchone()[0]).df()        
        df2 = con.sql(duckdb.sql(f" select min(sql_query) from results where nbr = {nbr} and model in ('qwen3:8b') ").fetchone()[0]).df()
        
        side_by_side = pd.concat([df1, df2], axis=1, keys=['DF1', 'DF2'])
        display(side_by_side)
    except Exception as e:
        print(f"Error executing query for nbr {nbr}: {e}")

question 1 : What is the overall total sales revenue?


,DF1,DF2
,total_sales,total_sales
0,5.138666e+09,5.138666e+09


question 2 : What is the total number of items sold across all transactions?


,DF1,DF2
,total_quantity,total_quantity
0,138963631.0,138963631.0


question 3 : What is the total monetary value of all returned items?


...

DF1                                 DF2            
              i_product_name total_sales          i_product_name total_sales
0               ableableable    96747.63            ableableable    96747.63
1           ableableableable   314629.92        ableableableable   314629.92
2      ableableableableought    95953.97   ableableableableought    95953.97
3     ableableableationought   349446.50  ableableableationought   349446.50
4       ableableablebarought   274503.21    ableableablebarought   274503.21
...                      ...         ...                     ...         ...
8869                    None   334795.48                    None   221840.40
8870                    None   385941.64                    None   206423.07
8871                    None   379536.63                    None   385941.64
8872                    None   225207.39                    None   237537.91
8873                    None   206423.07                    None    95714.25

[8874 rows x 4 columns]

question 16 : For each store, what was the percentage change in net sales between two consecutive recent years (e.g., 2001 and 2002), ordered alphabetically by store name.


DF1                                         DF2                  
  store_name percentage_change_in_net_sales s_store_name percentage_change
0       able                    -101.339547         able          2.122350
1      ation                    -101.128593        ation         -2.182444
2        bar                    -101.114776          bar          2.186394
3       eing                    -101.317246         eing         -0.271386
4        ese                    -101.325925          ese          0.784431
5      ought                    -101.378365        ought          1.695725

question 19 : For each item class, compare the average net sales value per sales transaction between preferred and non-preferred customers. Order by average net sales descending, and then by item class name alphabetically for ties.


DF1                                DF2                \
    item_class preferred avg_net_sales item_class customer_type   
0    wallpaper      None   2078.935543  wallpaper          None   
1        decor      None   2071.978736      decor          None   
2       stereo      None   2049.735175     stereo          None   
3      fishing      None   2049.724199    fishing          None   
4      science      None   2028.626098     custom          None   
..         ...       ...           ...        ...           ...   
295     sports      None   1684.019397   flatware          None   
296    camping      None   1681.420920    musical          None   
297     custom         Y   1680.005484     sports          None   
298     tennis      None   1672.502520    camping          None   
299   flatware      None   1672.412600     tennis          None   

                                   
    avg_net_sales_per_transaction  
0                     2182.882320  
1                     2172.952359  
2                     2145.389483  
3                     2100.967304  
4                     2078.424679  
..                            ...  
295                   1757.351243  
296                   1756.913914  
297                   1747.283095  
298                   1741.471667  
299                   1734.447058  

[300 rows x 6 columns]